# Download data and install dependencies

In [ ]:
!wget "https://docs.google.com/uc?export=download&id=12kStRhY4tlbjB8TUqQWwJrNkgrkNc_Fx&confirm=t" -O IMDB.csv

--2022-11-19 06:38:33--  https://docs.google.com/uc?export=download&id=15iOm6l8XrR-J4WUp0OhR2KTel1RAwebL&confirm=t
Resolving docs.google.com (docs.google.com)... 172.217.194.100, 172.217.194.101, 172.217.194.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.194.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-5o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7ovbhv6und6mq314m8qbhq0o5lhh3nbg/1668839850000/04375464899203764954/*/15iOm6l8XrR-J4WUp0OhR2KTel1RAwebL?e=download&uuid=f6dcceea-42f8-4079-b357-fee141f620d9 [following]
--2022-11-19 06:38:36--  https://doc-0o-5o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7ovbhv6und6mq314m8qbhq0o5lhh3nbg/1668839850000/04375464899203764954/*/15iOm6l8XrR-J4WUp0OhR2KTel1RAwebL?e=download&uuid=f6dcceea-42f8-4079-b357-fee141f620d9
Resolving doc-0o-5o-docs.googleusercontent.com (doc-0o-5o-docs.googleusercontent.com)... 142.251.12.1

In [ ]:
# !pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 19.0 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


# Import libraries

In [ ]:
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.legacy import data
from torchtext.legacy import datasets
import spacy
nlp = spacy.load('en_core_web_sm')

# Declare constants

In [ ]:
REVIEW = data.Field(tokenize='spacy',
                    tokenizer_language='en_core_web_sm',
                    include_lengths=True,
                    batch_first = True)
SENTIMENT = data.Field(pad_token=None,
                       unk_token=None,
                       dtype=torch.float)

MAX_VOCAB_SIZE = 25_000

EMBEDDING_DIM = 100
N_FILTERS = 150
FILTER_SIZES = [3,4,5]
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

# Prepare data

In [ ]:
fields = [("review", REVIEW), ("sentiment", SENTIMENT)]
full_data = data.TabularDataset(path='IMDB.csv',  
                                format='csv',
                                fields=fields,
                                skip_header=True)

In [ ]:
len(full_data)

50000

In [ ]:
train_data, valid_data, test_data = full_data.split(split_ratio=[0.8, 0.1, 0.1])

In [ ]:
len(train_data), len(valid_data), len(test_data)

(40000, 5000, 5000)

In [ ]:
REVIEW.build_vocab(train_data, 
                   max_size = MAX_VOCAB_SIZE,
                   vectors = "glove.6B.100d",
                   unk_init = torch.Tensor.normal_)
SENTIMENT.build_vocab(train_data)

INPUT_DIM = len(REVIEW.vocab)
PAD_IDX = REVIEW.vocab.stoi[REVIEW.pad_token]

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:17<00:00, 23407.72it/s]


In [ ]:
len(REVIEW.vocab), len(SENTIMENT.vocab)

(25002, 2)

In [ ]:
SENTIMENT.vocab.stoi

defaultdict(None, {'positive': 0, 'negative': 1})

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
                                                    (train_data,valid_data,test_data), 
                                                    batch_size=BATCH_SIZE,
                                                    sort_key=lambda x: len(x.review),
                                                    sort_within_batch=True,
                                                    device=device)

# Build the model

In [ ]:
class MultiChannelModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])

        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first = True)
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters + hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [batch size, sent len]
        
        # CNN ##################################################################

        embedded = self.embedding(text)    
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]   
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        
        conv_cat = torch.cat(pooled, dim = 1)
        #conv_cat = [batch size, n_filters * len(filter_sizes)]
        

        # LSTM #################################################################

        embedded = self.embedding(text)    
        #embedded = [batch size, sent len, emb dim]

        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, 
                                                            text_lengths.to('cpu'), 
                                                            batch_first=True)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #hidden = [batch size, num layers * num directions, hid dim]
        #cell = [batch size, num layers * num directions, hid dim]
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, 
                                                                  batch_first=True)
        #output = [batch size, sent len, hid dim * num directions]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        hidden_cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        #hidden_cat = [batch size, hid dim * num directions]

        result = self.dropout(torch.cat((conv_cat, hidden_cat), dim = 1))
        #result = [batch size, len(filter sizes) * n filters + hidden dim * 2]
            
        return self.fc(result)

In [ ]:
model = MultiChannelModel(INPUT_DIM, 
                          EMBEDDING_DIM, 
                          N_FILTERS,
                          FILTER_SIZES,
                          HIDDEN_DIM, 
                          OUTPUT_DIM, 
                          N_LAYERS, 
                          BIDIRECTIONAL, 
                          DROPOUT, 
                          PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,991,757 trainable parameters


## Use pretrained word embeddings

In [ ]:
pretrained_embeddings = REVIEW.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.0617e+00,  1.5777e-01,  7.4565e-01,  ...,  3.5641e-03,
         -1.4136e+00,  6.6208e-01],
        [ 8.1611e-01,  1.2886e+00, -3.2730e+00,  ..., -6.0458e-01,
          1.5436e+00,  1.7090e-01],
        [-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -1.4590e-01,
          8.2780e-01,  2.7062e-01],
        ...,
        [-2.5150e-02,  1.0131e+00, -6.5432e-01,  ...,  1.0351e+00,
          3.1442e-01,  1.4225e+00],
        [ 2.4607e-04,  1.5502e+00, -2.1343e-02,  ..., -5.4500e-01,
         -8.3147e-01, -7.3532e-01],
        [-2.5210e+00, -1.0057e-01, -5.4498e-01,  ..., -1.5410e+00,
          1.7243e+00,  9.3774e-01]])

In [ ]:
UNK_IDX = REVIEW.vocab.stoi[REVIEW.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -1.4590e-01,
          8.2780e-01,  2.7062e-01],
        ...,
        [-2.5150e-02,  1.0131e+00, -6.5432e-01,  ...,  1.0351e+00,
          3.1442e-01,  1.4225e+00],
        [ 2.4607e-04,  1.5502e+00, -2.1343e-02,  ..., -5.4500e-01,
         -8.3147e-01, -7.3532e-01],
        [-2.5210e+00, -1.0057e-01, -5.4498e-01,  ..., -1.5410e+00,
          1.7243e+00,  9.3774e-01]])


# Create optimzer and loss function

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

# Train the model

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.review
        
        predictions = model(text, text_lengths).squeeze(1)
        # print(batch.sentiment.squeeze(0).shape)
        # print(predictions.shape)
        loss = criterion(predictions, batch.sentiment.squeeze(0))
        
        acc = binary_accuracy(predictions, batch.sentiment.squeeze(0))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.review
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.sentiment.squeeze(0))
            
            acc = binary_accuracy(predictions, batch.sentiment.squeeze(0))

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 01 | Epoch Time: 1m 23s
	Train Loss: 0.495 | Train Acc: 74.71%
	 Val. Loss: 0.324 |  Val. Acc: 86.29%
Epoch: 02 | Epoch Time: 1m 24s
	Train Loss: 0.297 | Train Acc: 87.30%
	 Val. Loss: 0.281 |  Val. Acc: 88.11%
Epoch: 03 | Epoch Time: 1m 22s
	Train Loss: 0.196 | Train Acc: 92.31%
	 Val. Loss: 0.248 |  Val. Acc: 90.21%
Epoch: 04 | Epoch Time: 1m 22s
	Train Loss: 0.143 | Train Acc: 94.69%
	 Val. Loss: 0.259 |  Val. Acc: 90.76%
Epoch: 05 | Epoch Time: 1m 23s
	Train Loss: 0.095 | Train Acc: 96.55%
	 Val. Loss: 0.291 |  Val. Acc: 90.45%


In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.243 | Test Acc: 90.49%


# Test the model

In [ ]:
def predict_sentiment(model, sentence, min_len=5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    text_lengths = torch.Tensor([len(tokenized)])
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [REVIEW.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor, text_lengths))
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is terrible")

0.9987584352493286

In [ ]:
predict_sentiment(model, "This film is great")

0.006380683276802301

In [ ]:
predict_sentiment(model, "I don't like this film")

0.7619508504867554